In [ ]:
# Nibbling of GEAR/JRC rasters
Numbers = ["1","17","33","49","65","81","97","113","129","145","161","177",\
              "193","209","225","241","257","273","289","305","321","337","353"]
Folder = "D:\Data\Dropbox\Lactuca\Projects\AgLand SDR\Data\ESDAC_JRC\ESDAC R factor\R_Fractioned_16Days\R_JRC_Fraction_Day_"

for x in range(0,len(Numbers)):
    arcpy.Delete_management("ConRast")
    InFile = Folder + Numbers[x] + ".tif"
    OutFile = Folder + Numbers[x] + "Nibbled.tif"
    with arcpy.EnvManager(outputCoordinateSystem='PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]',\
                          cellSize="IsNullRFactorJRC(EU)Nibbled.tif"):
        # note by using cell size to 25x25 meters,extrapolations are based on nearest neighbour resampling, so no scale recalculations are done
        # INVest will do the same nearest neighbour but to 50x50meters. NOWHERE, the actual units change.
        out_raster = arcpy.sa.Con("IsNullRFactorJRC(EU)Nibbled.tif", 999, InFile, "Value = 1"); 
        out_raster.save("ConRast")
    with arcpy.EnvManager(cellSize="ConRast", compression="'LZW' 75"):
        out_raster = arcpy.sa.Nibble(InFile, "ConRast", "DATA_ONLY", "PROCESS_NODATA", None); 
        out_raster.save(OutFile)  

In [4]:
# calculate proportion avoided erosion per period
Numbers = ["1","17","33","49","65","81","97","113","129","145","161","177",\
              "193","209","225","241","257","273","289","305","321","337","353"]
Folder = "C:\DataDanny\AgLand SDR\Runs"
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]'):
    with arcpy.EnvManager(overwriteOutput=True):    
        for x in range(0,len(Numbers)):
            print(Numbers[x], end="\r") 
            InFile1 = Folder + "\\usle_JRC_Day" + Numbers[x] + ".tif"    
            InFile2 = Folder + "\\rkls_JRC_Day" + Numbers[x] + ".tif"  
            OutFile = Folder + "\\Prop_avoided_" + Numbers[x] + ".tif"
            out_raster = arcpy.sa.Divide(InFile1, InFile2); 
            out_raster.save("DivideRast")
            out_raster = arcpy.sa.Minus(1, "DivideRast"); 
            out_raster.save("Minus_Rast")
            out_raster = arcpy.sa.SetNull("LCM2015PlusCrops.tif", "Minus_Rast","BHT = '13. Saltwater' Or BHT = '14. Freshwater' Or BHT = '20. Urban' Or BHT = '21. SubUrban'"); 
            with arcpy.EnvManager(compression="'LZW'"):
                out_raster.save(OutFile)
            arcpy.Delete_management("DivideRast")
            arcpy.Delete_management("Minus_Rast")